In [1]:
include("../Engine/Engine.jl")
using .Engine

using StatsBase, DataFrames
using Graphs, SimpleWeightedGraphs
using Latexify
import PyPlot as plt
import Seaborn as sns
import PlotlyJS

rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["text.usetex"] = false
rcParams["font.family"] = ["sans-serif"]
rcParams["font.serif"] = ["Computer Modern Roman"];

WebIO._IJuliaInit()

In [2]:
graphs, cascades, df = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/COP26/data.jld2")
graphs2, cascades2, df2 = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/Random/data.jld2");

In [4]:
general_ranks = Metrics.get_general_ranks(COP26, cop_26_dates)
centrality_ranks = get_centrality_ranks(graphs, df, 0, "Any Edge");

In [ ]:
idx = 3
ranking = find_max_ranks(general_ranks[idx], centrality_ranks[idx])
show(ranking, allcols=true)

a = latexify(ranking, env=:array)
print(a)

In [ ]:
plot_correlation_matrices(general_ranks, centrality_ranks, 50, save=true, foldername="/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/")

In [ ]:
edge_type = "Any Edge"
N_biggest = 20
cuttoff = 0

between, between_actors = betweenness_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)
in_, in_actors = indegree_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)
out, out_actors = outdegree_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)

between_actors[3][1:N_biggest]

In [ ]:
plot_edge_types([graphs, graphs2], [df, df2], [0,0]; y="count_normalized", hue_order=["Before", "During", "After", "Control"],
    save=true, filename="/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/edge_type.pdf", log=true)


In [ ]:
plot_actors_per_level(cascades, df, control=cascades2, save=true, filename="/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/actor_level.pdf", inner_spacing=0)


In [ ]:
cascades[3]
PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...)
# PlotlyJS.savefig(PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...), "/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/cascade.pdf")